In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

import nltk

In [10]:
df = pd.read_excel('/kaggle/input/pontiaa/google-pontiac.xlsx')
print(df.shape)
df = df.head(500)
print(df.shape)

(657, 6)
(500, 6)


In [11]:
df.head()

,Unnamed: 0,Author,Date,Rating,Number,Reviews
0,NaN,Sanford Hejduk,2005-07-08 00:00:00,5,0,I am a DSM @ location 085. This was a role pla...
1,NaN,daddariotire,2017-09-15 16:12:14,1,5,I would not recommend Unifirst to anyone.Horri...
2,NaN,Carmen J Peña,2018-03-28 23:20:56,5,1,Love my job at National Accounts
3,NaN,William McGrath,2018-10-22 15:58:05,1,3,Decided we no longer needed their services at ...
4,NaN,Ann Ellis,2019-01-07 14:48:13,5,0,Great company to work for. You have the abilit...


In [12]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

sia = SentimentIntensityAnalyzer()

In [13]:
sia

In [14]:
import nltk  # Assuming you've already installed NLTK and downloaded the VADER lexicon
from tqdm import tqdm  # Assuming you have tqdm installed

sia = nltk.sentiment.vader.SentimentIntensityAnalyzer()

res = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    text = row['Reviews']
    myid = row['Author']

    # Ensure text is a string before sentiment analysis
    if not isinstance(text, str):
        text = str(text)

    res[myid] = sia.polarity_scores(text)

100%|██████████| 500/500 [00:00<00:00, 1356.09it/s]


In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Load your dataset with reviews and labels (1 for good service, 0 for not)
# df = pd.read_csv('reviews.csv')

# Preprocessing
# Clean the text, tokenize, and remove stopwords
def preprocess_text(text):
    # Implement your text preprocessing steps here
    return text

df['Processed Text'] = df['Reviews'].apply(preprocess_text)

# Feature Extraction
# Convert text data into numerical feature vectors using TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
X = tfidf_vectorizer.fit_transform(df['Processed Text'])
y = df['Rating']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Training
# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Model Evaluation
# Predict on the test set and evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Keyword Extraction
# Get feature names (words) from the TF-IDF vectorizer
feature_names = tfidf_vectorizer.get_feature_names_out()
# Get coefficients from the trained logistic regression model
coefficients = model.coef_[0]

# Get top keywords contributing to the positive class (good service)
top_keywords_indices = coefficients.argsort()[-10:]
top_keywords = [feature_names[idx] for idx in top_keywords_indices]
print("Top Keywords for Good Service:", top_keywords)

# Prediction
# Example: Predict whether a new review indicates good service
new_review = "The service was excellent and the staff was very helpful."
new_review_vectorized = tfidf_vectorizer.transform([new_review])
prediction = model.predict(new_review_vectorized)
if prediction[0] == 1:
    print("The review indicates good service.")
else:
    print("The review does not indicate good service.")


ValueError: np.nan is an invalid document, expected byte or unicode string.

In [ ]:
# Filter reviews praising the drivers (assuming 'Processed Text' column contains preprocessed text)
praising_driver_reviews = df[df['Reviews'].str.contains('driver')]

if not praising_driver_reviews.empty:
    # Convert filtered reviews into numerical feature vectors using TF-IDF
    X_praise_drivers = tfidf_vectorizer.transform(praising_driver_reviews['Processed Text'])

    # Predict sentiment using the logistic regression model
    y_pred_praise_drivers = model.predict(X_praise_drivers)

    # Extract keywords
    if y_pred_praise_drivers.any() == 1:  # Check if any reviews predict positive sentiment
        # Get coefficients from the trained logistic regression model
        coefficients_praise_drivers = model.coef_[0]

        # Get top keywords contributing to positive sentiment
        top_keywords_indices_praise_drivers = coefficients_praise_drivers.argsort()[-10:]
        top_keywords_praise_drivers = [feature_names[idx] for idx in top_keywords_indices_praise_drivers]
        print("Top Keywords for Praising Drivers:", top_keywords_praise_drivers)
    else:
        print("No reviews found praising the drivers.")
else:
    print("No reviews found mentioning drivers.")


In [ ]:
import spacy

# Load spaCy's English language model
nlp = spacy.load("en_core_web_sm")

# Example review praising someone
review = "Fabian Santo is always a friendly encounter and he gets our concerns completed in a timely manner when they arise. Keep up the excellent work Fabian from the service team at Carmax in Attleboro."

# Process the review using spaCy
doc = nlp(review)

# Extract proper nouns (persons) from the review
praised_persons = [entity.text for entity in doc.ents if entity.label_ == "PERSON"]

# Print the extracted persons (potential keywords indicating praise)
if praised_persons:
    print("Persons praised in the review:", praised_persons)
else:
    print("No persons praised in the review.")


In [ ]:
import spacy

# Load spaCy's English language model
nlp = spacy.load("en_core_web_sm")

# Example review praising someone
review = "Fabian Santo is always a friendly encounter and he gets our concerns completed in a timely manner when they arise. Keep up the excellent work Fabian from the service team at Carmax in Attleboro."

# Process the review using spaCy
doc = nlp(review)

# Extract adjectives from the review
adjectives = [token.text for token in doc if token.pos_ == "ADJ"]

# Print the extracted adjectives (potential keywords indicating praise)
if adjectives:
    print("Adjectives indicating praise in the review:", adjectives)
else:
    print("No adjectives indicating praise in the review.")


In [ ]:
import spacy
import pandas as pd

# Load spaCy's English language model
nlp = spacy.load("en_core_web_sm")

# Example DataFrame containing reviews
# Replace this with your actual DataFrame containing reviews
df = pd.DataFrame({
    'Reviews': [
        "Fabian Santo is always a friendly encounter and he gets our concerns completed in a timely manner when they arise. Keep up the excellent work Fabian from the service team at Carmax in Attleboro.",
        "The service provided by the team was exceptional. They were very professional and helpful.",
        "I had a wonderful experience with the service staff. They were courteous and efficient.",
        "The staff went above and beyond to assist me. I'm extremely satisfied with their service.",
        "The service was top-notch. The staff was attentive and friendly."
    ]
})

# Function to extract adjectives indicating praise from a review
def extract_praise_adjectives(review):
    # Process the review using spaCy
    doc = nlp(review)
    # Extract adjectives from the review
    adjectives = [token.text for token in doc if token.pos_ == "ADJ"]
    return adjectives

# Apply the function to each review in the DataFrame
df['Praise Adjectives'] = df['Reviews'].apply(extract_praise_adjectives)

# Print the DataFrame with extracted adjectives
print(df)


In [ ]:
import spacy
import pandas as pd

# Load spaCy's English language model
nlp = spacy.load("en_core_web_sm")

# Function to extract adjectives indicating praise from a review
def extract_praise_adjectives(review):
    if pd.isna(review):  # Check for NaN values
        return []
    # Process the review using spaCy
    doc = nlp(review)
    # Extract adjectives from the review
    adjectives = [token.text for token in doc if token.pos_ == "ADJ"]
    return adjectives

# Read reviews from an Excel sheet
df = pd.read_excel("/kaggle/input/pontiaa/google-pontiac.xlsx")

# Apply the function to each review in the DataFrame
df['Praise Adjectives'] = df['Reviews'].apply(extract_praise_adjectives)

# Print the DataFrame with extracted adjectives
print(df)


In [ ]:
# Generate a table with all reviews
all_reviews_table = df[['Author', 'Date', 'Rating', 'Reviews', 'Praise Adjectives']]

# Display the table
print(all_reviews_table)


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import spacy

# Load spaCy's English language model
nlp = spacy.load("en_core_web_sm")

# Assuming df is your DataFrame containing reviews and 'Processed Text' column contains preprocessed text
# Filter reviews mentioning drivers
praising_driver_reviews = df[df['Reviews'].str.contains('driver', case=False)]

if not praising_driver_reviews.empty:
    # Extract persons (names) praised in the reviews
    praised_persons = []
    for review in praising_driver_reviews['Reviews']:
        doc = nlp(review)
        persons = [entity.text for entity in doc.ents if entity.label_ == "PERSON"]
        praised_persons.extend(persons)

    # Convert persons list to lowercase for consistency
    praised_persons = [person.lower() for person in praised_persons]

    # Filter out duplicates
    praised_persons = list(set(praised_persons))

    if praised_persons:
        print("Persons praised in reviews mentioning drivers:", praised_persons)
    else:
        print("No persons praised in reviews mentioning drivers.")
else:
    print("No reviews found mentioning drivers.")


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Load the dataset
# Assuming df is your DataFrame containing the reviews and ratings

# Feature Extraction
# Convert text data into numerical feature vectors using TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
X = tfidf_vectorizer.fit_transform(df['Reviews'])
y = df['Rating']

# Model Training
# Train a logistic regression model
model = LogisticRegression()
model.fit(X, y)

# Get the vocabulary (feature names) from the TF-IDF vectorizer
feature_names = tfidf_vectorizer.get_feature_names_out()

# Get coefficients from the trained logistic regression model
coefficients = model.coef_[0]

# Get top keywords contributing to positive sentiment (rating 5)
top_keywords_indices = coefficients.argsort()[-10:]
top_keywords = [feature_names[idx] for idx in top_keywords_indices]
print("Top Keywords for Positive Sentiment (Rating 5):", top_keywords)


In [ ]:
# Predict sentiment for all reviews
y_pred = model.predict(X)

# Count the number of reviews classified as positive sentiment (assuming rating 5 indicates positive sentiment)
positive_reviews_count = sum(y_pred == 5)

print("Number of reviews based on good service (rating 5):", positive_reviews_count)


In [ ]:
import nltk
from tqdm import tqdm
import pandas as pd

vaders=pd.DataFrame(res).T
vaders = vaders.reset_index().rename(columns={'index': 'CustomerName'})

# Assuming you've already installed NLTK and downloaded the VADER lexicon
sia = nltk.sentiment.vader.SentimentIntensityAnalyzer()

res = {}

# Assuming 'wiI7pd' is the column containing text and 'd4r55' is the column containing years
for i, row in tqdm(df.iterrows(), total=len(df)):
    text = row['Reviews']
    myid = row['Author']
    year = row['Date']  # Assuming 'year' is the column containing the year information

    # Ensure text is a string before sentiment analysis
    if not isinstance(text, str):
        text = str(text)

    res[myid] = {'Date': year, **sia.polarity_scores(text)}

# Creating a DataFrame from the sentiment scores
vaders = pd.DataFrame(res).T.reset_index().rename(columns={'index': 'CustomerName'})
# Sorting the DataFrame based on the 'year' column in descending order
vaders_sorted = vaders.sort_values(by='Date', ascending=False)

# Displaying the sorted DataFrame
vaders_sorted.head()

In [ ]:
import nltk
from tqdm import tqdm
import pandas as pd


vaders=pd.DataFrame(res).T
vaders = vaders.reset_index().rename(columns={'index': 'CustomerName'})
# Assuming you've already installed NLTK and downloaded the VADER lexicon
sia = nltk.sentiment.vader.SentimentIntensityAnalyzer()

# Function to calculate sentiment scores
def calculate_sentiment(text):
    if not isinstance(text, str):
        text = str(text)
    return sia.polarity_scores(text)

# Assuming 'wiI7pd' is the column containing text, 'd4r55' is the column containing years, and 'year' is the column containing the year information
df['sentiment_scores'] = df['Reviews'].apply(calculate_sentiment)

# Extracting 'year' from the sentiment scores
df['Date'] = df['Date']  # Assuming 'year' is the column containing the year information

# Sorting the DataFrame based on the 'year' column in descending order
sorted_df = df.sort_values(by='Date', ascending=False)

# Displaying the sorted DataFrame
sorted_df[['Reviews', 'Author', 'Date']].head()

In [ ]:
positive_counts_by_year = df[df['sentiment_scores'].apply(lambda x: x['pos'] > x['neg'])].groupby('Date').size()


print("Positive Review Counts by Year:")
print(positive_counts_by_year)


In [ ]:
# Assuming you have already computed the DataFrame df with sentiment scores and year information

# Grouping the DataFrame by year and counting the number of negative reviews in each year
negative_counts_by_year = df[df['sentiment_scores'].apply(lambda x: x['neg'] > x['pos'])].groupby('Date').size()

# Printing the total number of negative reviews in each year
print("Negative Review Counts by Year:")
print(negative_counts_by_year)


In [ ]:
neutral_counts_by_year = df[df['sentiment_scores'].apply(lambda x: x['pos'] == x['neg'])].groupby('Date').size()

print("Neutral Review Counts by Year:")
print(neutral_counts_by_year)

In [ ]:
import nltk
import pandas as pd

# Assuming you've already installed NLTK and downloaded the VADER lexicon
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Initialize the VADER Sentiment Analyzer
sia = SentimentIntensityAnalyzer()

# Function to calculate sentiment scores
def calculate_sentiment(text):
    if not isinstance(text, str):
        text = str(text)
    return sia.polarity_scores(text)

# Assuming 'wiI7pd' is the column containing text, and 'year' is the column containing the year information
# You need to adjust the column names as per your DataFrame
df['sentiment_scores'] = df['Reviews'].apply(calculate_sentiment)

# Extracting 'year' from the sentiment scores
df['Date'] = df['Date'].dt.year  # Extract only the year from the datetime column

# Counting the number of reviews for each year
review_counts_by_year = df['Date'].value_counts()

# Sorting the years in descending order and obtaining the corresponding review counts
sorted_years = review_counts_by_year.sort_index(ascending=False)

# Splitting the DataFrame into positive, neutral, and negative reviews
positive_reviews = df[df['sentiment_scores'].apply(lambda x: x['pos'] > x['neg'])]
neutral_reviews = df[df['sentiment_scores'].apply(lambda x: x['pos'] == x['neg'])]
negative_reviews = df[df['sentiment_scores'].apply(lambda x: x['neg'] > x['pos'])]

# Counting the number of positive, neutral, and negative reviews for each year
positive_review_counts_by_year = positive_reviews['Date'].value_counts().reindex(sorted_years.index, fill_value=0)
neutral_review_counts_by_year = neutral_reviews['Date'].value_counts().reindex(sorted_years.index, fill_value=0)
negative_review_counts_by_year = negative_reviews['Date'].value_counts().reindex(sorted_years.index, fill_value=0)

# Creating a DataFrame to hold the review counts by year and sentiment
review_counts_df = pd.DataFrame({
    'Year': sorted_years.index,
    'Total Reviews': sorted_years.values,
    'Positive Reviews': positive_review_counts_by_year.values,
    'Neutral Reviews': neutral_review_counts_by_year.values,
    'Negative Reviews': negative_review_counts_by_year.values
})

# Displaying the DataFrame
print(review_counts_df)


In [ ]:
import nltk
import pandas as pd

from nltk.sentiment.vader import SentimentIntensityAnalyzer


sia = SentimentIntensityAnalyzer()


def calculate_sentiment(text):
    if not isinstance(text, str):
        text = str(text)
    return sia.polarity_scores(text)


def categorize_sentiment(Rating):
    if Rating > 4:
        return 'Positive'
    elif 2 < Rating <= 3:
        return 'Neutral'
    else:
        return 'Negative'


df['sentiment_scores'] = df['Reviews'].apply(calculate_sentiment)
df['Sentiment'] = df['Rating'].apply(categorize_sentiment)


df['Date'] = df['Date'].dt.year  

review_counts_by_year = df['Date'].value_counts()


sorted_years = review_counts_by_year.sort_index(ascending=False)


positive_reviews = df[df['Sentiment'] == 'Positive']
neutral_reviews = df[df['Sentiment'] == 'Neutral']
negative_reviews = df[df['Sentiment'] == 'Negative']

positive_review_counts_by_year = positive_reviews['Date'].value_counts().reindex(sorted_years.index, fill_value=0)
neutral_review_counts_by_year = neutral_reviews['Date'].value_counts().reindex(sorted_years.index, fill_value=0)
negative_review_counts_by_year = negative_reviews['Date'].value_counts().reindex(sorted_years.index, fill_value=0)


review_counts_df = pd.DataFrame({
    'Year': sorted_years.index,
    'Total Reviews': sorted_years.values,
    'Positive Reviews': positive_review_counts_by_year.values,
    'Neutral Reviews': neutral_review_counts_by_year.values,
    'Negative Reviews': negative_review_counts_by_year.values
})

# Displaying the DataFrame
print(review_counts_df)


In [ ]:

ratings_counts_by_year = df.groupby(['Date', 'Rating']).size().unstack(fill_value=0)
ratings_counts_by_year = ratings_counts_by_year.reindex(columns=range(1, 6), fill_value=0)

print(ratings_counts_by_year)


In [ ]:
import matplotlib.pyplot as plt

# Plotting the bar chart
plt.figure(figsize=(10, 6))

# Width of each bar
bar_width = 0.25

# Positions for the bars
years = range(len(review_counts_df))

# Plotting positive reviews
plt.bar([year - bar_width for year in years], review_counts_df['Positive Reviews'], bar_width, label='Positive', color='lightgreen')

# Plotting neutral reviews
plt.bar(years, review_counts_df['Neutral Reviews'], bar_width, label='Neutral', color='lightblue')

# Plotting negative reviews
plt.bar([year + bar_width for year in years], review_counts_df['Negative Reviews'], bar_width, label='Negative', color='lightcoral')

plt.title('Reviews by Year and Sentiment')
plt.xlabel('Year')
plt.ylabel('Number of Reviews')
plt.xticks(years, review_counts_df['Year'], rotation=45)
plt.legend(title='Sentiment', loc='upper left')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assuming you have already computed the review counts DataFrame named 'review_counts_df'

# Plotting the bar chart
plt.figure(figsize=(10, 6))

# Plotting positive reviews
plt.bar(review_counts_df['Year'], review_counts_df['Positive Reviews'], label='Positive', color='lightgreen')

# Plotting neutral reviews
plt.bar(review_counts_df['Year'], review_counts_df['Neutral Reviews'], bottom=review_counts_df['Positive Reviews'], label='Neutral', color='lightblue')

# Plotting negative reviews
plt.bar(review_counts_df['Year'], review_counts_df['Negative Reviews'], bottom=review_counts_df['Positive Reviews'] + review_counts_df['Neutral Reviews'], label='Negative', color='lightcoral')

plt.title('Reviews by Year and Sentiment')
plt.xlabel('Year')
plt.ylabel('Number of Reviews')
plt.xticks(rotation=45)
plt.legend(title='Sentiment', loc='upper left')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assuming you have already computed the review counts DataFrame named 'review_counts_df'

# Calculate the total number of reviews for each year
review_counts_df['Total'] = review_counts_df['Positive Reviews'] + review_counts_df['Neutral Reviews'] + review_counts_df['Negative Reviews']

# Calculate the percentage distribution of positive, neutral, and negative reviews for each year
review_counts_df['Positive (%)'] = (review_counts_df['Positive Reviews'] / review_counts_df['Total']) * 100
review_counts_df['Neutral (%)'] = (review_counts_df['Neutral Reviews'] / review_counts_df['Total']) * 100
review_counts_df['Negative (%)'] = (review_counts_df['Negative Reviews'] / review_counts_df['Total']) * 100

# Plotting the pie chart for each year
for year in review_counts_df['Year']:
    year_data = review_counts_df[review_counts_df['Year'] == year]
    labels = ['Positive', 'Neutral', 'Negative']
    sizes = [year_data.iloc[0]['Positive (%)'], year_data.iloc[0]['Neutral (%)'], year_data.iloc[0]['Negative (%)']]
    
    plt.figure(figsize=(6, 6))
    plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
    plt.title(f'Review Distribution for Year {year}')
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
    plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assuming you have already computed the review counts DataFrame named 'review_counts_df'

# Plotting the scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(review_counts_df['Year'], review_counts_df['Total Reviews'], color='blue', marker='o')
plt.title('Total Reviews by Year')
plt.xlabel('Year')
plt.ylabel('Total Number of Reviews')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have already computed the review counts DataFrame named 'review_counts_df'

# Set up the seaborn style
sns.set(style="white", color_codes=True)

# Create a pairplot with marginal histograms
g = sns.pairplot(review_counts_df, 
                 x_vars=['Positive Reviews', 'Neutral Reviews', 'Negative Reviews'], 
                 y_vars=['Total Reviews'],
                 height=5, aspect=1.5)

# Set labels and titles
plt.suptitle('Marginal Histograms for Positive, Neutral, and Negative Reviews vs Total Reviews', y=1.02)
g.set_axis_labels('Number of Positive Reviews', 'Total Number of Reviews', fontsize=12)

plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have already computed the review counts DataFrame named 'review_counts_df'

# Compute the correlation matrix
correlation_matrix = review_counts_df[['Positive Reviews', 'Neutral Reviews', 'Negative Reviews', 'Total Reviews']].corr()

# Set up the seaborn style
sns.set(style="white")

# Create the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title('Correlation Matrix of Review Counts')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assuming you have already computed the review counts DataFrame named 'review_counts_df'

# Initialize the subplot
fig, axes = plt.subplots(len(review_counts_df), 1, figsize=(6, 6 * len(review_counts_df)), sharex=True)

# Plotting pie charts and creating tables
for i, (year, data) in enumerate(review_counts_df.iterrows()):
    labels = ['Positive', 'Neutral', 'Negative']
    sizes = [data['Positive (%)'], data['Neutral (%)'], data['Negative (%)']]
    
    # Plotting pie chart
    ax = axes[i]
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
    ax.set_title(f'Review Distribution for Year {year}')
    ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
    
    # Creating table
    table_data = [[f'{size:.1f}%' for size in sizes]]
    ax.table(cellText=table_data, colLabels=labels, loc='bottom', cellLoc='center')

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assuming you have already computed the review counts DataFrame named 'review_counts_df'

# Combine the percentage data into a single DataFrame
percentage_data = review_counts_df[['Positive (%)', 'Neutral (%)', 'Negative (%)']]

# Create the table
fig, ax = plt.subplots(figsize=(10, 6))
table = ax.table(cellText=percentage_data.values,
                 colLabels=percentage_data.columns,
                 rowLabels=review_counts_df.index.astype(str),
                 loc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.2)  # Adjust table size

# Hide axes
ax.axis('off')

# Set title
ax.set_title('Percentage Distribution of Reviews by Year')

plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assuming you have already computed the review counts DataFrame named 'review_counts_df'

# Set up the figure and axes
fig, ax = plt.subplots(figsize=(10, 6))

# Plotting the bar chart
bar_width = 0.25
index = review_counts_df.index.astype(str)
years = range(len(index))

# Plotting positive reviews
ax.bar(years, review_counts_df['Positive (%)'], bar_width, label='Positive', color='lightgreen')

# Plotting neutral reviews
ax.bar([year + bar_width for year in years], review_counts_df['Neutral (%)'], bar_width, label='Neutral', color='lightblue')

# Plotting negative reviews
ax.bar([year + 2*bar_width for year in years], review_counts_df['Negative (%)'], bar_width, label='Negative', color='lightcoral')

# Set x-axis labels
ax.set_xticks([year + bar_width for year in years])
ax.set_xticklabels(index)

# Set labels and title
ax.set_xlabel('Year')
ax.set_ylabel('Percentage of Reviews')
ax.set_title('Percentage Distribution of Reviews by Year')
ax.legend()

# Show plot
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
